In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
print(torchvision.__version__)

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.11.0 True
0.12.0
0.22.1


In [3]:
# verify installation and cuda running
torch.zeros(1).cuda()

tensor([0.], device='cuda:0')

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.rc('font', size=12) 
plt.rc('figure', figsize = (12, 5))

# Settings for the visualizations
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2,'font.family': [u'times']})

import pandas as pd
pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 50)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [5]:
from torchvision import datasets, models, transforms
import torch.optim as optim

if torch.cuda.is_available():
    torch.cuda.init()
    print(torch.cuda.current_device())
    print(torch.cuda.device_count())
    print(torch.cuda.memory_allocated())
    print(torch.cuda.memory_cached())
    cuda = torch.device('cuda')
    cuda0 = torch.device('cuda:0')
    print(cuda0)

0
1
512
2097152
cuda:0


/home/user/anaconda3/envs/python37/lib/python3.7/site-packages/torch/cuda/memory.py:386: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


In [6]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |     512 B  |    3072 B  |    5632 B  |    5120 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |     512 B  |    3072 B  |    5632 B  |    5120 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |     512 B  |    3072 B  |    5632 B  |    5120 B  |\n|       from large pool |       0 B  |       0 B  |

In [7]:
torch.cuda.empty_cache()

In [8]:
from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

In [9]:
import os.path as osp
import numpy as np
from PIL import Image
import skimage
from skimage import io
print(skimage.__version__)

0.18.3


In [10]:
import mmcv

In [11]:
mmcv.__version__

'1.4.8'

In [12]:
classes = ['background', 'human']
palette = [[0,0,0], [255,0,0]]
class_weight = [1.0, 6.0] # 0.10 human pixels

In [13]:
import os.path as osp
import mmcv
import numpy as np
from PIL import Image

from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset


@DATASETS.register_module()
class HumanDataSet(CustomDataset):
    CLASSES = classes
    PALETTE = palette

    def __init__(self, **kwargs):
        super(HumanDataSet, self).__init__(**kwargs)
        assert osp.exists(self.img_dir)

    def results2img(self, results, imgfile_prefix, to_label_id, indices=None):
        if indices is None:
            indices = list(range(len(self)))

        mmcv.mkdir_or_exist(imgfile_prefix)
        result_files = []
        for result, idx in zip(results, indices):

            filename = self.img_infos[idx]['filename']
            basename = osp.splitext(osp.basename(filename))[0]

            png_filename = osp.join(imgfile_prefix, f'{basename}.png')

            output = Image.fromarray(result.astype(np.uint8))
            output.save(png_filename)
            result_files.append(png_filename)

        return result_files

    def format_results(self,
                       results,
                       imgfile_prefix,
                       to_label_id=True,
                       indices=None):
        
        if indices is None:
            indices = list(range(len(self)))

        assert isinstance(results, list), 'results must be a list.'
        assert isinstance(indices, list), 'indices must be a list.'

        result_files = self.results2img(results, imgfile_prefix, to_label_id,
                                        indices)
        return result_files

In [14]:
from mmcv import Config
cfg = Config.fromfile('../configs/segmenter/segmenter_vit-s_mask_8x1_512x512_160k_ade20k.py')

In [15]:
data_root = '../data/NewBinaryDataSet/binaryAnnotationsMIX30_v3_WithAddedNoHumanImages/'
img_dir = 'humanImages'
ann_dir = 'binaryAnnotations'

In [16]:
decode_head_loss_weight = 1.0
auxiliary_head_loss_weight = decode_head_loss_weight*0.4

In [17]:
norm_config_type = 'LN'

In [18]:
from mmseg.apis import set_random_seed
# model settings
cfg.backbone_norm_cfg = dict(type=norm_config_type, eps=1e-6, requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained=None,
    backbone=dict(
        type='VisionTransformer',
        img_size=(512, 512),
        patch_size=16,
        in_channels=3,
        embed_dims=384,
        num_layers=12,
        num_heads=6,
        drop_path_rate=0.1,
        attn_drop_rate=0.0,
        drop_rate=0.0,
        final_norm=True,
        norm_cfg=cfg.backbone_norm_cfg,
        with_cls_token=True,
        interpolate_mode='bicubic',
    ),
    decode_head=dict(
        type='SegmenterMaskTransformerHead',
        in_channels=384,
        channels=384,
        num_classes=2,
        num_layers=2,
        num_heads=6,
        embed_dims=384,
        dropout_ratio=0.0,
        loss_decode=dict( # Config of loss function for the decode_head.
                type='CrossEntropyLoss', # Type of loss used for segmentation.
                loss_name='loss_ce',
                use_sigmoid=False, # Whether use sigmoid activation for segmentation.
                loss_weight=decode_head_loss_weight, # Loss weight of auxiliary head, which is usually 0.4 of decode head.
                class_weight=class_weight
        ),
    ),
    test_cfg=dict(mode='slide', crop_size=(512, 512), stride=(480, 480)),
)


In [19]:
############### start dataset 
cfg.dataset_type = 'HumanDataSet'  # Dataset type, this will be used to define the dataset.
cfg.data_root = data_root  # Root path of data.
cfg.img_norm_cfg = dict(  # Image normalization config to normalize the input images.
    mean=[123.675, 116.28, 103.53],  # Mean values used to pre-training the pre-trained backbone models.
    std=[58.395, 57.12, 57.375],  # Standard variance used to pre-training the pre-trained backbone models.
    to_rgb=True)  # The channel orders of image used to pre-training the pre-trained backbone models.
cfg.crop_size = (512, 512)  # The crop size during training.

cfg.train_pipeline = [  # Training pipeline.
    dict(type='LoadImageFromFile'),  # First pipeline to load images from file path.
    dict(type='LoadAnnotations', reduce_zero_label=False),  # Second pipeline to load annotations for current image.
    dict(type='Resize',  # Augmentation pipeline that resize the images and their annotations.
        img_scale=(2048, 512),  # The largest scale of image.
        ratio_range=(0.5, 2.0)), # The augmented scale range as ratio.
    dict(type='RandomCrop',  # Augmentation pipeline that randomly crop a patch from current image.
        crop_size=cfg.crop_size,  # The crop size of patch.
        cat_max_ratio=0.75),  # The max area ratio that could be occupied by single category.
    dict(
        type='RandomFlip',  # Augmentation pipeline that flip the images and their annotations
        flip_ratio=0.5),  # The ratio or probability to flip
    dict(type='PhotoMetricDistortion'),  # Augmentation pipeline that distort current image with several photo metric methods.
    dict(
        type='Normalize',  # Augmentation pipeline that normalize the input images
        **cfg.img_norm_cfg),
    dict(type='Pad',  # Augmentation pipeline that pad the image to specified size.
        size=cfg.crop_size,  # The output size of padding.
        pad_val=0,  # The padding value for image.
        seg_pad_val=255),  # The padding value of 'gt_semantic_seg'.
    dict(type='DefaultFormatBundle'),  # Default format bundle to gather data in the pipeline
    dict(type='Collect',  # Pipeline that decides which keys in the data should be passed to the segmentor
        keys=['img', 'gt_semantic_seg'])
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),  # First pipeline to load images from file path
    dict(
        type='MultiScaleFlipAug',  # An encapsulation that encapsulates the test time augmentations
        img_scale=(2048, 512),  # Decides the largest scale for testing, used for the Resize pipeline
        flip=False,  # Whether to flip images during testing
        transforms=[
            dict(type='Resize',  # Use resize augmentation
                 keep_ratio=True),  # Whether to keep the ratio between height and width, the img_scale set here will be suppressed by the img_scale set above.
            #dict(type='ResizeToMultiple', size_divisor=32),
            dict(type='RandomFlip'),  # Thought RandomFlip is added in pipeline, it is not used when flip=False
            dict(
                type='Normalize',  # Normalization config, the values are from img_norm_cfg
                 **cfg.img_norm_cfg),
            dict(type='ImageToTensor', # Convert image to tensor
                keys=['img']),
            dict(type='Collect', # Collect pipeline that collect necessary keys for testing.
                keys=['img'])
        ]
    )
]
cfg.data = dict(
    samples_per_gpu=1,
    workers_per_gpu=8,
    train=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        img_dir=f'{img_dir}/training',
        ann_dir=f'{ann_dir}/training',
        pipeline=cfg.train_pipeline),
    val=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        img_dir=f'{img_dir}/validation',
        ann_dir=f'{ann_dir}/validation',
        pipeline=cfg.test_pipeline),
    test=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        img_dir=f'{img_dir}/validation',
        ann_dir=f'{ann_dir}/validation',
        pipeline=cfg.test_pipeline))
############ end dataset

In [20]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor

# Build the dataset
datasets = [build_dataset(cfg.data.train)]
#print(datasets[0].CLASSES)
print(len(datasets[0].CLASSES))

2022-04-14 11:17:03,914 - mmseg - INFO - Loaded 43675 images


2


In [21]:
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = len(datasets[0].CLASSES)

In [22]:
# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/test.segmentor.CAD.30e.5es.S'

############ start default runtime
cfg.log_config = dict(  # config to register logger hook
    interval=100,  # Interval to print the log
    hooks=[
        dict(type='TensorboardLoggerHook'),  # The Tensorboard logger is also supported
        dict(type='TextLoggerHook', by_epoch=True)
    ]
)
cfg.dist_params = dict(backend='nccl')  # Parameters to setup distributed training, the port can also be set.
cfg.log_level = 'INFO'  # The level of logging.
cfg.load_from = None #'./work_dirs/testv1.cleaned_augmented_dataset.160000iter.earlystoping/best_mIoU_epoch_4.pth'  # load models as a pre-trained model from a given path. This will not resume training.
cfg.resume_from = None #'./work_dirs/testv1.cleaned_augmented_dataset.160000iter.earlystoping/best_mIoU_epoch_4.pth'  # Resume checkpoints from a given path, the training will be resumed from the iteration when the checkpoint's is saved.
cfg.workflow = [('train', 1)]  # , ('val', 1) Workflow for runner. [('train', 1)] means there is only one workflow and the workflow named 'train' is executed once. The workflow trains the model by 40000 iterations according to the `runner.max_iters`.
cfg.cudnn_benchmark = True  # Whether use cudnn_benchmark to speed up, which is fast for fixed input size.
############ end default runtime

############ start schedule
cfg.optimizer = dict(
    type='SGD',
    lr=0.01,
    weight_decay=0.0005,
    momentum=0.9
)
cfg.optimizer_config = dict()  # Config used to build the optimizer hook, refer to https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/optimizer.py#L8 for implementation details.
cfg.lr_config = dict(
    policy='poly',  # The policy of scheduler, also support Step, CosineAnnealing, Cyclic, etc. Refer to details of supported LrUpdater from https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/lr_updater.py#L9.
    by_epoch=True,
    warmup='linear',
    warmup_iters=1,
    warmup_ratio=1e-6,
    warmup_by_epoch=True,
    power=0.9,
    min_lr=1e-4
)
# velocity = Momentum(velocity) + gradient(1-momentum)
#cfg.momentum_config = dict(
#    policy='LinearAnnealing',
#    min_momentum=0.5,
#    by_epoch=True,
#    warmup='linear',
#    warmup_iters=1,
#    warmup_ratio=0.9
#)
cfg.runner=dict(
    type='EpochBasedRunner', # Type of runner to use (i.e. IterBasedRunner or EpochBasedRunner)
    max_epochs=50
)
cfg.checkpoint_config = dict(  # Config to set the checkpoint hook, Refer to https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/checkpoint.py for implementation.
    by_epoch=True,  # Whether count by epoch or not.
    interval=1,
    meta=dict(
        CLASSES=classes,
        PALETTE=palette
    )
)
cfg.evaluation = dict(  # The config to build the evaluation hook. Please refer to mmseg/core/evaluation/eval_hook.py for details.
    interval=1,  # The interval of evaluation.
    metric='mIoU',
    pre_eval=True,
    early_stopping=True,
    max_epochs=5
)

############ end schedule

In [23]:
# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
checkpoint = 'https://download.openmmlab.com/mmsegmentation/v0.5/pretrain/segmenter/vit_small_p16_384_20220308-410f6037.pth'
backbone_norm_cfg = dict(type='LN', eps=1e-06, requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained=
    'https://download.openmmlab.com/mmsegmentation/v0.5/pretrain/segmenter/vit_small_p16_384_20220308-410f6037.pth',
    backbone=dict(
        type='VisionTransformer',
        img_size=(512, 512),
        patch_size=16,
        in_channels=3,
        embed_dims=384,
        num_layers=12,
        num_heads=6,
        drop_path_rate=0.1,
        attn_drop_rate=0.0,
        drop_rate=0.0,
        final_norm=True,
        norm_cfg=dict(type='LN', eps=1e-06, requires_grad=True),
        with_cls_token=True,
        interpolate_mode='bicubic'),
    decode_head=dict(
        type='SegmenterMaskTransformerHead',
        in_channels=384,
        channels=384,
        num_classes=2,
        num_layers=2,
        num_heads=6,
        embed_d

In [24]:
# Build the detector
model = build_segmentor(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

/home/user/workspace/github/mmsegmentation/mmseg/models/backbones/vit.py:193: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '


In [25]:
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES
model.PALETTE = datasets[0].PALETTE

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

In [26]:
train_segmentor(model, datasets, cfg, distributed=False, validate=True, meta=dict())

2022-04-14 11:17:13,845 - mmseg - INFO - Loaded 4855 images
2022-04-14 11:17:13,846 - mmseg - INFO - Start running, host: user@ESWS-VT02-01, work_dir: /home/user/workspace/github/mmsegmentation/demo/work_dirs/test.segmentor.CAD.30e.5es.S
2022-04-14 11:17:13,846 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TensorboardLoggerHook              
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TensorboardLoggerHook              
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) I

2022-04-14 11:22:05,170 - mmseg - INFO - Epoch [1][2800/43675]	lr: 6.409e-04, eta: 2 days, 15:01:02, time: 0.104, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4275, decode.acc_seg: 82.9183, loss: 0.4275
2022-04-14 11:22:15,556 - mmseg - INFO - Epoch [1][2900/43675]	lr: 6.638e-04, eta: 2 days, 15:00:39, time: 0.104, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4333, decode.acc_seg: 82.6785, loss: 0.4333
2022-04-14 11:22:25,930 - mmseg - INFO - Epoch [1][3000/43675]	lr: 6.867e-04, eta: 2 days, 15:00:08, time: 0.104, data_time: 0.001, memory: 1060, decode.loss_ce: 0.3948, decode.acc_seg: 84.9357, loss: 0.3948
2022-04-14 11:22:36,325 - mmseg - INFO - Epoch [1][3100/43675]	lr: 7.096e-04, eta: 2 days, 14:59:53, time: 0.104, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4576, decode.acc_seg: 80.8030, loss: 0.4576
2022-04-14 11:22:46,713 - mmseg - INFO - Epoch [1][3200/43675]	lr: 7.325e-04, eta: 2 days, 14:59:34, time: 0.104, data_time: 0.001, memory: 1060, decode.loss_ce: 0.3955

2022-04-14 11:29:00,944 - mmseg - INFO - Epoch [1][6800/43675]	lr: 1.557e-03, eta: 2 days, 14:52:21, time: 0.104, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4329, decode.acc_seg: 81.9049, loss: 0.4329
2022-04-14 11:29:16,384 - mmseg - INFO - Epoch [1][6900/43675]	lr: 1.580e-03, eta: 2 days, 15:18:34, time: 0.154, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4248, decode.acc_seg: 83.3407, loss: 0.4248
2022-04-14 11:29:38,100 - mmseg - INFO - Epoch [1][7000/43675]	lr: 1.603e-03, eta: 2 days, 16:16:40, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.3740, decode.acc_seg: 85.7502, loss: 0.3740
2022-04-14 11:29:59,815 - mmseg - INFO - Epoch [1][7100/43675]	lr: 1.625e-03, eta: 2 days, 17:13:08, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.3814, decode.acc_seg: 85.5792, loss: 0.3814
2022-04-14 11:30:21,536 - mmseg - INFO - Epoch [1][7200/43675]	lr: 1.648e-03, eta: 2 days, 18:08:03, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.3967

2022-04-14 11:43:22,781 - mmseg - INFO - Epoch [1][10800/43675]	lr: 2.473e-03, eta: 3 days, 15:40:42, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4282, decode.acc_seg: 82.6246, loss: 0.4282
2022-04-14 11:43:44,478 - mmseg - INFO - Epoch [1][10900/43675]	lr: 2.495e-03, eta: 3 days, 16:04:18, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.3918, decode.acc_seg: 85.0689, loss: 0.3918
2022-04-14 11:44:06,172 - mmseg - INFO - Epoch [1][11000/43675]	lr: 2.518e-03, eta: 3 days, 16:27:25, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4547, decode.acc_seg: 81.5416, loss: 0.4547
2022-04-14 11:44:27,875 - mmseg - INFO - Epoch [1][11100/43675]	lr: 2.541e-03, eta: 3 days, 16:50:10, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4471, decode.acc_seg: 81.3540, loss: 0.4471
2022-04-14 11:44:49,603 - mmseg - INFO - Epoch [1][11200/43675]	lr: 2.564e-03, eta: 3 days, 17:12:34, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0

2022-04-14 11:57:51,444 - mmseg - INFO - Epoch [1][14800/43675]	lr: 3.388e-03, eta: 4 days, 3:13:30, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4243, decode.acc_seg: 82.2352, loss: 0.4243
2022-04-14 11:58:13,169 - mmseg - INFO - Epoch [1][14900/43675]	lr: 3.411e-03, eta: 4 days, 3:25:59, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4189, decode.acc_seg: 83.1743, loss: 0.4189
2022-04-14 11:58:34,878 - mmseg - INFO - Epoch [1][15000/43675]	lr: 3.434e-03, eta: 4 days, 3:38:14, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4177, decode.acc_seg: 83.3359, loss: 0.4177
2022-04-14 11:58:56,595 - mmseg - INFO - Epoch [1][15100/43675]	lr: 3.457e-03, eta: 4 days, 3:50:22, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4021, decode.acc_seg: 83.4612, loss: 0.4021
2022-04-14 11:59:18,324 - mmseg - INFO - Epoch [1][15200/43675]	lr: 3.480e-03, eta: 4 days, 4:02:21, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4242

...

2022-04-14 13:24:10,045 - mmseg - INFO - Epoch [1][38800/43675]	lr: 8.884e-03, eta: 4 days, 20:57:07, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.3860, decode.acc_seg: 84.4361, loss: 0.3860
2022-04-14 13:24:31,734 - mmseg - INFO - Epoch [1][38900/43675]	lr: 8.906e-03, eta: 4 days, 20:58:41, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4954, decode.acc_seg: 77.5467, loss: 0.4954
2022-04-14 13:24:53,431 - mmseg - INFO - Epoch [1][39000/43675]	lr: 8.929e-03, eta: 4 days, 21:00:14, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4341, decode.acc_seg: 81.5315, loss: 0.4341
2022-04-14 13:25:15,130 - mmseg - INFO - Epoch [1][39100/43675]	lr: 8.952e-03, eta: 4 days, 21:01:48, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.3646, decode.acc_seg: 86.0889, loss: 0.3646
2022-04-14 13:25:36,814 - mmseg - INFO - Epoch [1][39200/43675]	lr: 8.975e-03, eta: 4 days, 21:03:20, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0

2022-04-14 13:38:06,252 - mmseg - INFO - Epoch [1][42800/43675]	lr: 9.799e-03, eta: 4 days, 21:26:35, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4167, decode.acc_seg: 83.4996, loss: 0.4167
2022-04-14 13:38:27,943 - mmseg - INFO - Epoch [1][42900/43675]	lr: 9.822e-03, eta: 4 days, 21:27:52, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.3867, decode.acc_seg: 85.2665, loss: 0.3867
2022-04-14 13:38:49,634 - mmseg - INFO - Epoch [1][43000/43675]	lr: 9.845e-03, eta: 4 days, 21:29:09, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.4019, decode.acc_seg: 84.0723, loss: 0.4019
2022-04-14 13:39:11,317 - mmseg - INFO - Epoch [1][43100/43675]	lr: 9.868e-03, eta: 4 days, 21:30:25, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0.3982, decode.acc_seg: 84.0627, loss: 0.3982
2022-04-14 13:39:33,010 - mmseg - INFO - Epoch [1][43200/43675]	lr: 9.891e-03, eta: 4 days, 21:31:41, time: 0.217, data_time: 0.001, memory: 1060, decode.loss_ce: 0

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4855/4855, 6.7 task/s, elapsed: 723s, ETA:     0s

2022-04-14 13:53:19,279 - mmseg - INFO - per class results:
2022-04-14 13:53:19,280 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 89.11 | 100.0 |
|   human    |  0.0  |  0.0  |
+------------+-------+-------+
2022-04-14 13:53:19,280 - mmseg - INFO - Summary:
2022-04-14 13:53:19,280 - mmseg - INFO - 
+-------+-------+------+
|  aAcc |  mIoU | mAcc |
+-------+-------+------+
| 89.11 | 44.55 | 50.0 |
+-------+-------+------+
2022-04-14 13:53:19,517 - mmseg - INFO - Now best checkpoint is saved as best_mIoU_epoch_1.pth.
2022-04-14 13:53:19,517 - mmseg - INFO - Best mIoU is 0.4455 at 1 epoch.
2022-04-14 13:53:19,517 - mmseg - INFO - early stopping reset to 5
2022-04-14 13:53:19,518 - mmseg - INFO - Early stopping: False
2022-04-14 13:53:19,523 - mmseg - INFO - Epoch(val) [1][4855]	aAcc: 0.8911, mIoU: 0.4455, mAcc: 0.5000, IoU.background: 0.8911, IoU.human: 0.0000, Acc.background: 1.0000, Acc.human: 0.0000
2022-04

KeyboardInterrupt: 

In [ ]:
# human IoU above 70%


In [ ]:
img = mmcv.imread('../data/ade/ADEChallengeData2016/images-binary/validation/ADE_val_00000011.jpg')
plt.imshow(img)

In [ ]:
img = mmcv.imread('../data/ade/ADEChallengeData2016/images-binary/validation/ADE_val_00000011.jpg')

model.cfg = cfg
result = inference_segmentor(model, img)

In [ ]:
np.asarray(result).max()

In [ ]:
plt.figure(figsize=(8, 6))
show_result_pyplot(model, img, result, palette)

In [ ]:
img = mmcv.imread('../data/ade/ADEChallengeData2016/binary-labels/validation/ADE_val_00000011.png')
img[img == 1] = 255
plt.imshow(img, cmap="gray")

In [ ]:
img.max()

In [ ]:
img.min()

In [ ]:
# print('show video')
# test a video and show the results
video = mmcv.VideoReader('../data/ramp.mp4')

In [ ]:
import time

In [ ]:
video[1].shape

In [ ]:
x = 100

In [ ]:
%%time
start = time.time()

for frame in video[:x]: # time - how many frame per sec - metrics acc - IoU
    result = inference_segmentor(model, frame)
    #model.show_result(frame, result)

end = time.time()

In [ ]:
from humanfriendly import format_timespan
aux0 = (end-start)
print("Time:", format_timespan(aux0))
aux1 = aux0/x
print("Time/frame:", format_timespan(aux1))
print("FPS: ", 1/aux1)

In [ ]:
# display the training loss changes
